In [9]:
import pandas as pd
import numpy as np
import joblib
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANT ENGINE v18 (DYNAMIC FEATURE EXTRACTION)    ")
print("=======================================================")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
BANKROLL = 30.00        
MIN_EDGE = 0.015          # 1.5% Minimum Edge
MAX_ODDS_CAP = 6.00       # Cap to avoid lottery tickets
MAX_STAKE_PCT = 0.12      # Max 12% bet size
KELLY_FRACTION = 0.25     # Quarter Kelly

# --- FIXTURES ---
# Add your fixtures here. Format: Home, Away, Odds (1, X, 2)
fixtures = [
    {"Home": "Manchester United", "Away": "Newcastle United", "Odds_1": 2.53, "Odds_X": 3.61, "Odds_2": 2.68},
    {"Home": "Nottingham Forest", "Away": "Manchester City", "Odds_1": 5.29, "Odds_X": 4.45, "Odds_2": 1.58},
    {"Home": "Arsenal",           "Away": "Brighton & Hove Albion", "Odds_1": 1.40, "Odds_X": 4.92, "Odds_2": 7.85},
    {"Home": "Brentford",         "Away": "Bournemouth",      "Odds_1": 2.29, "Odds_X": 3.60, "Odds_2": 3.03},
    {"Home": "Burnley",           "Away": "Everton",          "Odds_1": 4.01, "Odds_X": 3.35, "Odds_2": 2.01},
    {"Home": "Liverpool",         "Away": "Wolverhampton Wanderers", "Odds_1": 1.24, "Odds_X": 6.59, "Odds_2": 11.28},
    {"Home": "West Ham United",   "Away": "Fulham",           "Odds_1": 2.68, "Odds_X": 3.41, "Odds_2": 2.63},
    {"Home": "Chelsea",           "Away": "Aston Villa",      "Odds_1": 1.85, "Odds_X": 3.88, "Odds_2": 4.03},
    {"Home": "Sunderland",        "Away": "Leeds United",     "Odds_1": 2.58, "Odds_X": 3.23, "Odds_2": 2.86},
    {"Home": "Crystal Palace",    "Away": "Tottenham Hotspur", "Odds_1": 2.29, "Odds_X": 3.30, "Odds_2": 3.25},
]

# ==============================================================================
# 2. LOAD MODEL & DATA
# ==============================================================================
try:
    artifacts = joblib.load('football_model_final.pkl')
    model = artifacts['model']
    features = artifacts['features']
    current_elos = artifacts['elo_dict']
    df_recent = artifacts['df_recent']
    print(f"✅ Model Loaded. Features expected: {len(features)}")
except FileNotFoundError:
    print("❌ Error: 'football_model_final.pkl' not found.")
    exit()

def get_stats(team):
    # Get last known stats for a team
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    
    # DYNAMIC EXTRACTION
    # We pull ANY column that starts with the team prefix and contains 'roll_'
    # This automatically grabs all 60+ player stats you added.
    stats = {}
    valid_cols = [c for c in df_recent.columns if 'roll_' in c]
    
    for col in valid_cols:
        if col.startswith(prefix):
            # Clean name: home_roll_players_SCA -> roll_players_SCA
            generic_name = col.replace(prefix, '')
            stats[generic_name] = last[col]
            
    return stats

# ==============================================================================
# 3. ANALYSIS LOOP
# ==============================================================================
opportunities = []
ELITE_TEAMS = ["Manchester City", "Liverpool", "Arsenal", "Real Madrid", "Barcelona"]

print(f"\n🔍 ANALYZING {len(fixtures)} MATCHES...")

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    h_stats, a_stats = get_stats(h_team), get_stats(a_team)
    h_elo, a_elo = current_elos.get(h_team, 1500), current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats: continue

    # --- INPUT CONSTRUCTION ---
    input_data = {}
    
    # 1. Base Features
    input_data['diff_elo'] = (h_elo + 60) - a_elo # Sync with Training (+60)
    input_data['home_elo'] = h_elo
    input_data['away_elo'] = a_elo
    input_data['diff_rest'] = 0
    
    # 2. Dynamic Features (Rolling & Diffs)
    # The model expects specific column names. We must match them.
    # We iterate through the MODEL'S feature list to ensure we don't miss anything.
    
    for feat in features:
        if feat in input_data: continue # Skip ELO/Rest
        
        # Handle Rolling
        if 'home_roll_' in feat:
            key = feat.replace('home_', '') # e.g. roll_players_SCA
            input_data[feat] = h_stats.get(key, 0)
        elif 'away_roll_' in feat:
            key = feat.replace('away_', '')
            input_data[feat] = a_stats.get(key, 0)
            
        # Handle Differentials
        elif 'diff_' in feat:
            # Reconstruct the key from diff_
            # diff_players_SCA -> roll_players_SCA (This is the tricky part)
            # Your training script created diffs by subtracting home_roll - away_roll
            key = feat.replace('diff_', 'roll_')
            input_data[feat] = h_stats.get(key, 0) - a_stats.get(key, 0)

    # 3. Predict
    # Fill missing with 0 (safe for sparse player stats)
    input_df = pd.DataFrame([input_data]).reindex(columns=features, fill_value=0)
    probs = model.predict_proba(input_df)[0]
    p_away, p_draw, p_home = probs[0], probs[1], probs[2]

    # --- LOGIC GATES ---
    
    # A. Deadlock Check
    is_tight_game = abs(p_home - p_away) < 0.12
    if is_tight_game:
        # Boost Draw for analysis
        shift = 0.08
        p_draw_adj = p_draw + shift
        p_home_adj = p_home - (shift/2)
        p_away_adj = p_away - (shift/2)
    else:
        p_draw_adj, p_home_adj, p_away_adj = p_draw, p_home, p_away

    # B. Goliath Check
    block_underdog = False
    if (a_team in ELITE_TEAMS and h_elo < a_elo - 200) or (h_team in ELITE_TEAMS and a_elo < h_elo - 200):
        block_underdog = True

    print(f"{h_team[:10]} vs {a_team[:10]}: H {p_home:.2f} | D {p_draw:.2f} | A {p_away:.2f} {'[Tight]' if is_tight_game else ''}")

    # --- VALUE FINDER ---
    bets_for_match = []
    
    for outcome, p, odds, label in [
        ('HOME', p_home_adj, f['Odds_1'], h_team),
        ('DRAW', p_draw_adj, f['Odds_X'], 'Draw'),
        ('AWAY', p_away_adj, f['Odds_2'], a_team)
    ]:
        if odds > MAX_ODDS_CAP: continue
        
        # Goliath Safety
        if block_underdog:
            is_favorite = (p == max(p_home, p_draw, p_away))
            if not is_favorite: continue
            
        edge = (p * odds) - 1
        min_p = 0.23 if outcome == 'DRAW' else 0.20
        
        if edge > MIN_EDGE and p > min_p:
            bets_for_match.append({'Type': outcome, 'Team': label, 'Odds': odds, 'Prob': p, 'Edge': edge})

    # --- SELECTION STRATEGY ---
    if bets_for_match:
        # Sort by Edge
        best = sorted(bets_for_match, key=lambda x: x['Edge'], reverse=True)[0]
        
        # Deadlock Override: If Tight Game AND Draw is Valid, Prefer Draw unless Winner Edge is massive
        if is_tight_game:
            draw_bet = next((b for b in bets_for_match if b['Type'] == 'DRAW'), None)
            if draw_bet:
                if best['Edge'] < (draw_bet['Edge'] + 0.15):
                    best = draw_bet
                    best['Reason'] = "Safety Draw"
        
        best['Match'] = f"{h_team} vs {a_team}"
        opportunities.append(best)

# ==============================================================================
# 4. PORTFOLIO
# ==============================================================================
if opportunities:
    res = pd.DataFrame(opportunities)
    
    def calculate_stake(row):
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        f = ((b * p) - q) / b
        return max(0, min(f * KELLY_FRACTION, MAX_STAKE_PCT))

    res['Stake_Pct'] = res.apply(calculate_stake, axis=1)
    res['Stake'] = res['Stake_Pct'] * BANKROLL
    res['Profit'] = res['Stake'] * (res['Odds'] - 1)
    
    res = res[res['Stake'] > 0.01].sort_values('Stake', ascending=False)
    
    # Display
    display_df = res[['Match', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Stake', 'Profit']].copy()
    display_df.columns = ['Match', 'Bet Type', 'Selection', 'Odds', 'Win %', 'Edge', 'Stake ($)', 'Profit ($)']
    
    styled = (display_df.style
        .format({'Odds': '{:.2f}', 'Win %': '{:.1%}', 'Edge': '{:.1%}', 'Stake ($)': '${:.2f}', 'Profit ($)': '${:.2f}'})
        .background_gradient(subset=['Win %'], cmap='Greens', vmin=0.2, vmax=0.8)
        .background_gradient(subset=['Edge'], cmap='Blues', vmin=0.0, vmax=0.5)
        .bar(subset=['Stake ($)'], color='#ff6b6b', vmin=0)
        .bar(subset=['Profit ($)'], color='#51cf66', vmin=0)
        .set_caption(f"🚀 FINAL QUANT SLIP v18 (Player Stats) | Bankroll: ${BANKROLL}")
    )
    display(styled)
    
    print(f"\n💰 Total Risk: ${res['Stake'].sum():.2f}")
else:
    print("\n📉 No Value Bets Found.")

   🤖 QUANT ENGINE v18 (DYNAMIC FEATURE EXTRACTION)    
✅ Model Loaded. Features expected: 268

🔍 ANALYZING 10 MATCHES...
Manchester vs Newcastle : H 0.41 | D 0.23 | A 0.36 [Tight]
Nottingham vs Manchester: H 0.45 | D 0.09 | A 0.46 [Tight]
Arsenal vs Brighton &: H 0.65 | D 0.04 | A 0.31 
Brentford vs Bournemout: H 0.39 | D 0.22 | A 0.39 [Tight]
Burnley vs Everton: H 0.47 | D 0.09 | A 0.44 [Tight]
Liverpool vs Wolverhamp: H 0.81 | D 0.02 | A 0.17 
West Ham U vs Fulham: H 0.25 | D 0.32 | A 0.42 
Chelsea vs Aston Vill: H 0.54 | D 0.25 | A 0.21 
Sunderland vs Leeds Unit: H 0.28 | D 0.42 | A 0.30 [Tight]
Crystal Pa vs Tottenham : H 0.66 | D 0.09 | A 0.25 


,Match,Bet Type,Selection,Odds,Win %,Edge,Stake ($),Profit ($)
6,Crystal Palace vs Tottenham Hotspur,HOME,Crystal Palace,2.29,65.9%,51.0%,$2.97,$3.83
1,Nottingham Forest vs Manchester City,HOME,Nottingham Forest,5.29,41.2%,118.0%,$2.06,$8.85
5,Sunderland vs Leeds United,DRAW,Draw,3.23,49.6%,60.3%,$2.03,$4.52
3,Burnley vs Everton,HOME,Burnley,4.01,42.8%,71.6%,$1.78,$5.37
4,West Ham United vs Fulham,AWAY,Fulham,2.63,42.3%,11.3%,$0.52,$0.85
0,Manchester United vs Newcastle United,DRAW,Draw,3.61,30.7%,11.0%,$0.32,$0.82
2,Brentford vs Bournemouth,DRAW,Draw,3.60,29.8%,7.3%,$0.21,$0.55



💰 Total Risk: $9.89


In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings
from scipy.stats import poisson
from IPython.display import display

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANT ENGINE (POISSON GOAL MODEL)                ")
print("=======================================================")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
BANKROLL = 30.00        
MIN_EDGE = 0.02           
MAX_ODDS_CAP = 6.00       
MAX_STAKE_PCT = 0.12      
KELLY_FRACTION = 0.25     

# --- FIXTURES ---
fixtures = [
    {"Home": "Manchester United", "Away": "Newcastle United", "Odds_1": 2.53, "Odds_X": 3.61, "Odds_2": 2.68},
    {"Home": "Nottingham Forest", "Away": "Manchester City", "Odds_1": 5.29, "Odds_X": 4.45, "Odds_2": 1.58},
    {"Home": "Arsenal",           "Away": "Brighton & Hove Albion", "Odds_1": 1.40, "Odds_X": 4.92, "Odds_2": 7.85},
    {"Home": "Brentford",         "Away": "Bournemouth",      "Odds_1": 2.29, "Odds_X": 3.60, "Odds_2": 3.03},
    {"Home": "Burnley",           "Away": "Everton",          "Odds_1": 4.01, "Odds_X": 3.35, "Odds_2": 2.01},
    {"Home": "Liverpool",         "Away": "Wolverhampton Wanderers", "Odds_1": 1.24, "Odds_X": 6.59, "Odds_2": 11.28},
    {"Home": "West Ham United",   "Away": "Fulham",           "Odds_1": 2.68, "Odds_X": 3.41, "Odds_2": 2.63},
    {"Home": "Chelsea",           "Away": "Aston Villa",      "Odds_1": 1.85, "Odds_X": 3.88, "Odds_2": 4.03},
    {"Home": "Sunderland",        "Away": "Leeds United",     "Odds_1": 2.58, "Odds_X": 3.23, "Odds_2": 2.86},
    {"Home": "Crystal Palace",    "Away": "Tottenham Hotspur", "Odds_1": 2.29, "Odds_X": 3.30, "Odds_2": 3.25},
]

# ==============================================================================
# 2. LOAD MODEL
# ==============================================================================
try:
    artifacts = joblib.load('football_model_final.pkl')
    
    # Check if it's actually the Poisson model
    if not artifacts.get('is_poisson', False):
        print("❌ Error: The saved model is NOT a Poisson model.")
        print("   Please re-run the 'football_train_poisson.py' script.")
        exit()
        
    model_home = artifacts['model_home']
    model_away = artifacts['model_away']
    features = artifacts['features']
    current_elos = artifacts['elo_dict']
    df_recent = artifacts['df_recent']
    print(f"✅ Poisson Model Loaded.")
except:
    print("❌ Error loading pickle.")
    exit()

def get_stats(team):
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    
    stats = {}
    for col in features:
        # We need to extract the raw values for the features
        if 'roll_' in col:
            key = col.replace('home_roll_', '').replace('away_roll_', '')
            if key not in stats:
                raw_col = f"{prefix}roll_{key}"
                stats[key] = last[raw_col] if raw_col in last else 0
    return stats

# ==============================================================================
# 3. ANALYSIS LOOP
# ==============================================================================
opportunities = []
print(f"\n🔍 ANALYZING {len(fixtures)} MATCHES...")

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    h_stats, a_stats = get_stats(h_team), get_stats(a_team)
    h_elo, a_elo = current_elos.get(h_team, 1500), current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats: continue

    # --- FEATURE VECTOR ---
    input_data = {
        'diff_elo': (h_elo + 70) - a_elo, 
        'home_elo': h_elo, 'away_elo': a_elo, 'diff_rest': 0, 
    }
    
    # Fill dynamic features
    for feat in features:
        if feat in input_data: continue
        if 'home_roll_' in feat:
            key = feat.replace('home_roll_', '')
            input_data[feat] = h_stats.get(key, 0)
        elif 'away_roll_' in feat:
            key = feat.replace('away_roll_', '')
            input_data[feat] = a_stats.get(key, 0)
        elif 'diff_' in feat:
            key = feat.replace('diff_', '')
            input_data[feat] = h_stats.get(key, 0) - a_stats.get(key, 0)

    # --- PREDICT EXPECTED GOALS (Lambda) ---
    input_df = pd.DataFrame([input_data]).reindex(columns=features, fill_value=0)
    
    lambda_home = model_home.predict(input_df)[0]
    lambda_away = model_away.predict(input_df)[0]
    
    # --- SIMULATE PROBABILITIES ---
    max_goals = 10
    home_probs = [poisson.pmf(i, lambda_home) for i in range(max_goals)]
    away_probs = [poisson.pmf(i, lambda_away) for i in range(max_goals)]
    matrix = np.outer(home_probs, away_probs)
    
    p_draw = np.trace(matrix)
    p_home = np.sum(np.tril(matrix, -1))
    p_away = np.sum(np.triu(matrix, 1))
    
    print(f"{h_team[:10]} vs {a_team[:10]}: xG {lambda_home:.2f}-{lambda_away:.2f} | H {p_home:.2f} D {p_draw:.2f} A {p_away:.2f}")

    # --- VALUE FINDER ---
    bets_for_match = []
    
    for outcome, p, odds, label in [
        ('HOME', p_home, f['Odds_1'], h_team),
        ('DRAW', p_draw, f['Odds_X'], 'Draw'),
        ('AWAY', p_away, f['Odds_2'], a_team)
    ]:
        if odds > MAX_ODDS_CAP: continue
        
        edge = (p * odds) - 1
        
        # Min Probability (Poisson is more precise, so we can lower the bar slightly)
        min_p = 0.20 
        
        if edge > MIN_EDGE and p > min_p:
            bets_for_match.append({'Type': outcome, 'Team': label, 'Odds': odds, 'Prob': p, 'Edge': edge})

    if bets_for_match:
        # Sort by Edge
        best = sorted(bets_for_match, key=lambda x: x['Edge'], reverse=True)[0]
        best['Match'] = f"{h_team} vs {a_team}"
        best['xG_Score'] = f"{lambda_home:.1f} - {lambda_away:.1f}"
        opportunities.append(best)

# ==============================================================================
# 4. PORTFOLIO
# ==============================================================================
if opportunities:
    res = pd.DataFrame(opportunities)
    
    def calculate_stake(row):
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        f = ((b * p) - q) / b
        return max(0, min(f * KELLY_FRACTION, MAX_STAKE_PCT))

    res['Stake_Pct'] = res.apply(calculate_stake, axis=1)
    res['Stake'] = res['Stake_Pct'] * BANKROLL
    res['Profit'] = res['Stake'] * (res['Odds'] - 1)
    
    res = res[res['Stake'] > 0.01].sort_values('Stake', ascending=False)
    
    display_df = res[['Match', 'xG_Score', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Stake', 'Profit']].copy()
    display_df.columns = ['Match', 'xG', 'Bet Type', 'Selection', 'Odds', 'Win %', 'Edge', 'Stake ($)', 'Profit ($)']
    
    styled = (display_df.style
        .format({'Odds': '{:.2f}', 'Win %': '{:.1%}', 'Edge': '{:.1%}', 'Stake ($)': '${:.2f}', 'Profit ($)': '${:.2f}'})
        .background_gradient(subset=['Win %'], cmap='Greens', vmin=0.2, vmax=0.8)
        .background_gradient(subset=['Edge'], cmap='Blues', vmin=0.0, vmax=0.5)
        .bar(subset=['Stake ($)'], color='#ff6b6b', vmin=0)
        .bar(subset=['Profit ($)'], color='#51cf66', vmin=0)
        .set_caption(f"🚀 POISSON QUANT SLIP | Bankroll: ${BANKROLL}")
    )
    display(styled)
    
    print(f"\n💰 Total Risk: ${res['Stake'].sum():.2f}")
else:
    print("\n📉 No Value Bets Found.")

   🤖 QUANT ENGINE (POISSON GOAL MODEL)                
✅ Poisson Model Loaded.

🔍 ANALYZING 10 MATCHES...
Manchester vs Newcastle : xG 1.59-1.23 | H 0.46 D 0.25 A 0.30
Nottingham vs Manchester: xG 1.43-1.50 | H 0.36 D 0.25 A 0.39
Arsenal vs Brighton &: xG 1.95-1.09 | H 0.57 D 0.22 A 0.21
Brentford vs Bournemout: xG 1.57-1.27 | H 0.44 D 0.25 A 0.31
Burnley vs Everton: xG 1.25-1.35 | H 0.35 D 0.26 A 0.39
Liverpool vs Wolverhamp: xG 2.12-0.97 | H 0.64 D 0.20 A 0.16
West Ham U vs Fulham: xG 1.07-1.61 | H 0.25 D 0.25 A 0.50
Chelsea vs Aston Vill: xG 1.68-1.30 | H 0.46 D 0.24 A 0.30
Sunderland vs Leeds Unit: xG 1.41-1.43 | H 0.37 D 0.25 A 0.38
Crystal Pa vs Tottenham : xG 1.66-1.02 | H 0.52 D 0.24 A 0.23


,Match,xG,Bet Type,Selection,Odds,Win %,Edge,Stake ($),Profit ($)
1,Nottingham Forest vs Manchester City,1.4 - 1.5,HOME,Nottingham Forest,5.29,36.1%,91.2%,$1.59,$6.84
4,West Ham United vs Fulham,1.1 - 1.6,AWAY,Fulham,2.63,49.8%,31.0%,$1.43,$2.33
7,Crystal Palace vs Tottenham Hotspur,1.7 - 1.0,HOME,Crystal Palace,2.29,52.2%,19.5%,$1.14,$1.47
3,Burnley vs Everton,1.2 - 1.3,HOME,Burnley,4.01,34.5%,38.4%,$0.96,$2.88
0,Manchester United vs Newcastle United,1.6 - 1.2,HOME,Manchester United,2.53,45.8%,15.8%,$0.78,$1.19
5,Chelsea vs Aston Villa,1.7 - 1.3,AWAY,Aston Villa,4.03,29.8%,20.2%,$0.50,$1.51
6,Sunderland vs Leeds United,1.4 - 1.4,AWAY,Leeds United,2.86,38.0%,8.5%,$0.34,$0.64
2,Arsenal vs Brighton & Hove Albion,1.9 - 1.1,DRAW,Draw,4.92,21.9%,7.5%,$0.14,$0.56



💰 Total Risk: $6.88


In [ ]:
!pip install pandas numpy scikit-learn scipy joblib ipython

In [1]:
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANT ENGINE (MULTI-MODEL CONSENSUS)             ")
print("=======================================================")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
BANKROLL = 30.00        
MIN_EDGE = 0.02           
MAX_ODDS_CAP = 6.00       
KELLY_FRACTION = 0.25     
MAX_STAKE_PCT = 0.12      

# --- FIXTURES ---
fixtures = [
    {"Home": "Manchester United", "Away": "Newcastle United", "Odds_1": 2.53, "Odds_X": 3.61, "Odds_2": 2.68},
    {"Home": "Nottingham Forest", "Away": "Manchester City", "Odds_1": 5.29, "Odds_X": 4.45, "Odds_2": 1.58},
    {"Home": "Arsenal",           "Away": "Brighton & Hove Albion", "Odds_1": 1.40, "Odds_X": 4.92, "Odds_2": 7.85},
    {"Home": "Brentford",         "Away": "Bournemouth",      "Odds_1": 2.29, "Odds_X": 3.60, "Odds_2": 3.03},
    {"Home": "Burnley",           "Away": "Everton",          "Odds_1": 4.01, "Odds_X": 3.35, "Odds_2": 2.01},
    {"Home": "Liverpool",         "Away": "Wolverhampton Wanderers", "Odds_1": 1.24, "Odds_X": 6.59, "Odds_2": 11.28},
    {"Home": "West Ham United",   "Away": "Fulham",           "Odds_1": 2.68, "Odds_X": 3.41, "Odds_2": 2.63},
    {"Home": "Chelsea",           "Away": "Aston Villa",      "Odds_1": 1.85, "Odds_X": 3.88, "Odds_2": 4.03},
    {"Home": "Sunderland",        "Away": "Leeds United",     "Odds_1": 2.58, "Odds_X": 3.23, "Odds_2": 2.86},
    {"Home": "Crystal Palace",    "Away": "Tottenham Hotspur", "Odds_1": 2.29, "Odds_X": 3.30, "Odds_2": 3.25},
]

# ==============================================================================
# 2. LOAD BOTH MODELS
# ==============================================================================
try:
    # 1. Main Model (XGB/RF)
    main_artifacts = joblib.load('football_model_final.pkl')
    model_main = main_artifacts['model']
    features_main = main_artifacts['features']
    current_elos = main_artifacts['elo_dict']
    df_recent = main_artifacts['df_recent']
    
    # 2. GRU Model
    model_gru = tf.keras.models.load_model("football_gru_model.keras")
    scaler_gru = joblib.load("gru_scaler.pkl")
    history_gru = joblib.load("team_history.pkl") # You must have saved this in training!
    
    print(f"✅ Main Model & GRU Network Loaded.")
except Exception as e:
    print(f"❌ Error loading models: {e}")
    print("Ensure you ran BOTH training scripts.")
    exit()

# Helper for Main Stats
def get_main_stats(team):
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    stats = {}
    for col in features_main:
        if 'roll_' in col:
            key = col.replace('home_roll_', '').replace('away_roll_', '')
            if key not in stats:
                raw_col = f"{prefix}roll_{key}"
                stats[key] = last[raw_col] if raw_col in last else 0
    return stats

# Helper for GRU Sequences
def get_gru_seq(team):
    hist = history_gru.get(team, [])
    if len(hist) < 5: return np.zeros((1, 5, 5)) # Pad if missing
    seq = np.array(hist[-5:])
    return scaler_gru.transform(seq).reshape(1, 5, 5)

# ==============================================================================
# 3. ANALYSIS LOOP
# ==============================================================================
opportunities = []
ELITE_TEAMS = ["Manchester City", "Liverpool", "Arsenal"]

print(f"\n🔍 ANALYZING {len(fixtures)} MATCHES...")

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    
    # --- MAIN MODEL PREDICTION ---
    h_stats = get_main_stats(h_team)
    a_stats = get_main_stats(a_team)
    h_elo = current_elos.get(h_team, 1500)
    a_elo = current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats: continue

    input_main = {
        'diff_elo': (h_elo + 70) - a_elo, 
        'home_elo': h_elo, 'away_elo': a_elo, 'diff_rest': 0, 
    }
    # Fill rolling
    for feat in features_main:
        if 'home_roll_' in feat:
            key = feat.replace('home_roll_', '')
            input_main[feat] = h_stats.get(key, 0)
        elif 'away_roll_' in feat:
            key = feat.replace('away_roll_', '')
            input_main[feat] = a_stats.get(key, 0)
        elif 'diff_' in feat and feat not in ['diff_elo', 'diff_rest']:
            key = feat.replace('diff_', '')
            input_main[feat] = h_stats.get(key, 0) - a_stats.get(key, 0)

    df_main = pd.DataFrame([input_main]).reindex(columns=features_main, fill_value=0)
    probs_main = model_main.predict_proba(df_main)[0] # [Away, Draw, Home]
    
    # --- GRU MODEL PREDICTION ---
    seq_h = get_gru_seq(h_team)
    seq_a = get_gru_seq(a_team)
    # Static input for GRU (ELO Diff Scaled)
    static_gru = np.array([[(h_elo + 70 - a_elo)/400.0]])
    
    probs_gru = model_gru.predict([seq_h, seq_a, static_gru], verbose=0)[0] # [Away, Draw, Home]

    # --- CONSENSUS LOGIC ---
    # We average the probabilities, but give Main Model 70% weight (Acc 56%) 
    # and GRU 30% weight (Acc 47%).
    # HOWEVER, for Draws, we respect GRU more.
    
    p_final = (probs_main * 0.7) + (probs_gru * 0.3)
    p_away, p_draw, p_home = p_final[0], p_final[1], p_final[2]
    
    # BONUS: If GRU Draw Prob is > 35%, boost Final Draw Prob significantly
    if probs_gru[1] > 0.35:
        p_draw += 0.05
        p_home -= 0.025
        p_away -= 0.025

    print(f"{h_team[:10]} vs {a_team[:10]}: H {p_home:.2f} | D {p_draw:.2f} | A {p_away:.2f} [Hybrid]")

    # --- VALUE FINDER ---
    bets_for_match = []
    
    for outcome, p, odds, label in [
        ('HOME', p_home, f['Odds_1'], h_team),
        ('DRAW', p_draw, f['Odds_X'], 'Draw'),
        ('AWAY', p_away, f['Odds_2'], a_team)
    ]:
        if odds > MAX_ODDS_CAP: continue
        
        # Goliath Filter
        if (a_team in ELITE_TEAMS and h_elo < a_elo - 200) and outcome == 'HOME': continue
        if (h_team in ELITE_TEAMS and a_elo < h_elo - 200) and outcome == 'AWAY': continue

        edge = (p * odds) - 1
        
        # Min Probability
        min_p = 0.23 if outcome == 'DRAW' else 0.20
        
        if edge > MIN_EDGE and p > min_p:
            bets_for_match.append({'Type': outcome, 'Team': label, 'Odds': odds, 'Prob': p, 'Edge': edge})

    if bets_for_match:
        best = sorted(bets_for_match, key=lambda x: x['Edge'], reverse=True)[0]
        
        # DEADLOCK LOGIC (Using Consensus)
        gap = abs(p_home - p_away)
        if gap < 0.10:
            draw_bet = next((b for b in bets_for_match if b['Type'] == 'DRAW'), None)
            # If Main Model OR GRU strongly suggests Draw, take it
            if draw_bet and (probs_gru[1] > 0.30 or probs_main[1] > 0.28):
                if best['Edge'] < (draw_bet['Edge'] + 0.15):
                    best = draw_bet
                    best['Reason'] = "Consensus Draw"
        
        best['Match'] = f"{h_team} vs {a_team}"
        opportunities.append(best)

# ==============================================================================
# 4. PORTFOLIO
# ==============================================================================
if opportunities:
    res = pd.DataFrame(opportunities)
    
    def calculate_stake(row):
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        f = ((b * p) - q) / b
        return max(0, min(f * KELLY_FRACTION, MAX_STAKE_PCT))

    res['Stake_Pct'] = res.apply(calculate_stake, axis=1)
    res['Stake'] = res['Stake_Pct'] * BANKROLL
    res['Profit'] = res['Stake'] * (res['Odds'] - 1)
    
    res = res[res['Stake'] > 0.01].sort_values('Stake', ascending=False)
    
    display_df = res[['Match', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Stake', 'Profit']].copy()
    display_df.columns = ['Match', 'Bet Type', 'Selection', 'Odds', 'Win %', 'Edge', 'Stake ($)', 'Profit ($)']
    
    styled = (display_df.style
        .format({'Odds': '{:.2f}', 'Win %': '{:.1%}', 'Edge': '{:.1%}', 'Stake ($)': '${:.2f}', 'Profit ($)': '${:.2f}'})
        .background_gradient(subset=['Win %'], cmap='Greens', vmin=0.2, vmax=0.8)
        .background_gradient(subset=['Edge'], cmap='Blues', vmin=0.0, vmax=0.5)
        .bar(subset=['Stake ($)'], color='#ff6b6b', vmin=0)
        .bar(subset=['Profit ($)'], color='#51cf66', vmin=0)
        .set_caption(f"🚀 MULTI-MODEL QUANT SLIP | Bankroll: ${BANKROLL}")
    )
    display(styled)
    
    print(f"\n💰 Total Risk: ${res['Stake'].sum():.2f}")
else:
    print("\n📉 No Value Bets Found.")

ModuleNotFoundError: No module named 'pandas'